# Assignment 2

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./05_src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [23]:

# Load the libraries as required.
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import pickle

In [2]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


# Get X and Y

Create the features data frame and target data.

In [3]:
#Create the features data frame 
X = fires_dt.drop(columns=['area'])

In [4]:
#Create target data
y = fires_dt['area']

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [5]:
# Identify columns
numeric_cols = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
categorical_cols = ['month', 'day']

# Preproc1: Simple scaling + one-hot
preproc1 = ColumnTransformer([
    ('num', StandardScaler(), numeric_cols),
    ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_cols)
])

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [6]:
from sklearn.preprocessing import FunctionTransformer

# Add log transform to one numeric column (e.g., rain)
log_transformer = FunctionTransformer(np.log1p, validate=True)

preproc2 = ColumnTransformer([
    ('num', StandardScaler(), numeric_cols),
    ('log', log_transformer, ['rain']),
     ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_cols)
])

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [7]:
# Pipeline A = preproc1 + baseline
from sklearn.linear_model import LinearRegression

# Pipeline A: preproc1 + LinearRegression (baseline)
pipe_a = Pipeline([
    ('preprocessing', preproc1),
    ('regressor', LinearRegression())
])

In [8]:
# Pipeline B = preproc2 + baseline
from sklearn.neighbors import KNeighborsRegressor

# Pipeline B: preproc2 + KNeighborsRegressor (baseline)
pipe_b = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', KNeighborsRegressor())
])

In [9]:
# Pipeline C = preproc1 + advanced model
from sklearn.ensemble import RandomForestRegressor

# Pipeline C: preproc1 + RandomForestRegressor (advanced)
pipe_c = Pipeline([
    ('preprocessing', preproc1),
    ('regressor', RandomForestRegressor(random_state=42))
])

In [10]:
# Pipeline D = preproc2 + advanced model

# Pipeline D: preproc2 + RandomForestRegressor (advanced)
pipe_d = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', RandomForestRegressor(random_state=42))
])
    

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [11]:
from sklearn.model_selection import GridSearchCV


In [12]:
# Grid for Pipeline A (LinearRegression)
param_grid_a = {
    'regressor__fit_intercept': [True, False]
}

In [13]:
# Grid for Pipeline B (KNeighborsRegressor)
param_grid_b = {
    'regressor__n_neighbors': [3, 5, 7, 9]
}


In [14]:
# Grid for Pipeline C and D (RandomForest)
param_grid_cd = {
    'regressor__n_estimators': [50, 100, 150, 200]
}

In [15]:
# Create GridSearchCV objects
grid_a = GridSearchCV(pipe_a, param_grid_a, cv=5, scoring='neg_mean_squared_error')
grid_b = GridSearchCV(pipe_b, param_grid_b, cv=5, scoring='neg_mean_squared_error')
grid_c = GridSearchCV(pipe_c, param_grid_cd, cv=5, scoring='neg_mean_squared_error')
grid_d = GridSearchCV(pipe_d, param_grid_cd, cv=5, scoring='neg_mean_squared_error')

In [16]:
# Fit all grids
grids = {'A': grid_a, 'B': grid_b, 'C': grid_c, 'D': grid_d}
for name, grid in grids.items():
    grid.fit(X, y)
    print(f"Pipeline {name} best score: {grid.best_score_:.3f}")
    print(f"Best params: {grid.best_params_}")

c:\Users\anika\OneDrive\Documents\DSI_cohort8\Module-Production\production\production-env\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\anika\OneDrive\Documents\DSI_cohort8\Module-Production\production\production-env\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\anika\OneDrive\Documents\DSI_cohort8\Module-Production\production\production-env\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\anika\OneDrive\Documents\DSI_cohort8\Module-Production\production\production-env\Lib\site-packages\sklearn\preprocessing\

Pipeline A best score: -4351.577
Best params: {'regressor__fit_intercept': False}


c:\Users\anika\OneDrive\Documents\DSI_cohort8\Module-Production\production\production-env\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\anika\OneDrive\Documents\DSI_cohort8\Module-Production\production\production-env\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\anika\OneDrive\Documents\DSI_cohort8\Module-Production\production\production-env\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\anika\OneDrive\Documents\DSI_cohort8\Module-Production\production\production-env\Lib\site-packages\sklearn\preprocessing\

Pipeline B best score: -4962.743
Best params: {'regressor__n_neighbors': 9}


c:\Users\anika\OneDrive\Documents\DSI_cohort8\Module-Production\production\production-env\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\anika\OneDrive\Documents\DSI_cohort8\Module-Production\production\production-env\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\anika\OneDrive\Documents\DSI_cohort8\Module-Production\production\production-env\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\anika\OneDrive\Documents\DSI_cohort8\Module-Production\production\production-env\Lib\site-packages\sklearn\preprocessing\

Pipeline C best score: -4777.080
Best params: {'regressor__n_estimators': 200}


c:\Users\anika\OneDrive\Documents\DSI_cohort8\Module-Production\production\production-env\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\anika\OneDrive\Documents\DSI_cohort8\Module-Production\production\production-env\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\anika\OneDrive\Documents\DSI_cohort8\Module-Production\production\production-env\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\anika\OneDrive\Documents\DSI_cohort8\Module-Production\production\production-env\Lib\site-packages\sklearn\preprocessing\

Pipeline D best score: -4770.624
Best params: {'regressor__n_estimators': 200}


# Evaluate

+ Which model has the best performance?

Based on the cross-validation scores, Pipeline A (preproc1 + LinearRegression) performed best with a mean squared error of -4351.6. This was better than the other pipelines, including the tree-based models.

The results were surprising to me - I expected RandomForest to do better with non-linear relationships. But for this forest fire dataset, the simple linear model with basic scaling worked best. The log transformation on rain in preproc2 didn't help, and kNN performed worse, possibly because the data doesn't have clear local patterns.

The negative MSE scores are confusing (more negative is worse), but Pipeline A has the highest (least negative) score, making it the best performer.

# Export

+ Save the best performing model to a pickle file.

In [17]:
import pickle

# Get the best model (Pipeline A's best estimator)
best_model = grid_a.best_estimator_

# Save to pickle file
with open('best_forest_fire_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

print("Best model saved as 'best_forest_fire_model.pkl'")
print(f"Model type: {type(best_model)}")
print(f"Model parameters: {best_model.get_params()}")

Best model saved as 'best_forest_fire_model.pkl'
Model type: <class 'sklearn.pipeline.Pipeline'>
Model parameters: {'memory': None, 'steps': [('preprocessing', ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                  'isi', 'temp', 'rh', 'wind', 'rain']),
                                ('cat',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore'),
                                 ['month', 'day'])])), ('regressor', LinearRegression(fit_intercept=False))], 'transform_input': None, 'verbose': False, 'preprocessing': ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                  'isi', 'temp', 'rh', 'wind', 'rain']),
                                ('cat',
                                 OneHotE

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [28]:
# Get feature importance from coefficients
preprocessor = best_model.named_steps['preprocessing']
linear_model = best_model.named_steps['regressor']
feature_names = preprocessor.get_feature_names_out()
coefficients = linear_model.coef_

feature_importance = pd.DataFrame({
    'feature': feature_names,
    'coefficient': coefficients,
    'abs_importance': np.abs(coefficients)
}).sort_values('abs_importance', ascending=False)

print("Feature importance from LinearRegression coefficients:")
print(feature_importance.head(10))

Feature importance from LinearRegression coefficients:
           feature  coefficient  abs_importance
20  cat__month_sep    25.191071       25.191071
19  cat__month_oct    21.648670       21.648670
18  cat__month_nov   -21.602108       21.602108
16  cat__month_mar   -21.514640       21.514640
15  cat__month_jun   -21.167932       21.167932
4          num__dc   -19.636418       19.636418
11  cat__month_dec    18.433478       18.433478
22    cat__day_sat    18.086568       18.086568
3         num__dmc    10.702017       10.702017
12  cat__month_feb   -10.592787       10.592787


In [29]:
# 1. Local explanation for a specific observation
test_idx = 10
prediction = best_model.predict(X.iloc[[test_idx]])

print(f"1. Local explanation for observation {test_idx}:")
print(f"   Predicted burned area: {prediction[0]:.2f} ha")
print("   Most influential features for this prediction:")
print("   - September (month_sep): +25.19 (increases prediction)")
print("   - October (month_oct): +21.65 (increases prediction)")
print("   - DC index (num__dc): -19.64 (decreases prediction)")

# 2. Global feature importance
print(f"\n2. Global feature importance (all training data):")
print("   Most important features:")
print("   1. September month (+25.19)")
print("   2. October month (+21.65)")
print("   3. November month (-21.60)")
print("   4. March month (-21.51)")
print("   5. DC index (-19.64)")
print("\n   Least important features are those with smallest coefficients,")
print("   likely some day-of-week encodings and minor weather variables.")

# 3. Feature removal strategy
print(f"\n3. Feature removal strategy:")
print("   Remove features with |coefficient| < 1.0 (low impact).")
print("   Test using nested cross-validation:")
print("   - Train model with reduced feature set")
print("   - Compare MSE to full model")
print("   - If performance similar, keep simpler model")
print("   Month features dominate, suggesting temporal patterns are key.")

1. Local explanation for observation 10:
   Predicted burned area: 37.31 ha
   Most influential features for this prediction:
   - September (month_sep): +25.19 (increases prediction)
   - October (month_oct): +21.65 (increases prediction)
   - DC index (num__dc): -19.64 (decreases prediction)

2. Global feature importance (all training data):
   Most important features:
   1. September month (+25.19)
   2. October month (+21.65)
   3. November month (-21.60)
   4. March month (-21.51)
   5. DC index (-19.64)

   Least important features are those with smallest coefficients,
   likely some day-of-week encodings and minor weather variables.

3. Feature removal strategy:
   Remove features with |coefficient| < 1.0 (low impact).
   Test using nested cross-validation:
   - Train model with reduced feature set
   - Compare MSE to full model
   - If performance similar, keep simpler model
   Month features dominate, suggesting temporal patterns are key.


*(Answer here.)*

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at the `help` channel. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.